[View in Colaboratory](https://colab.research.google.com/github/tattn/TuriCreatePlayground/blob/master/TuriCreate_ImageClassifier.ipynb)

In [2]:
!pip install turicreate

# Google Driveのマウント

In [3]:
# https://qiita.com/uni-3/items/201aaa2708260cc790b8
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

gpg: keybox '/tmp/tmpqcxyqcrf/pubring.gpg' created
gpg: /tmp/tmpqcxyqcrf/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
··········


In [4]:
!mkdir -p drive
!google-drive-ocamlfuse drive
!mkdir -p drive/colab/ImageClassifier/training_data # 学習用のデータを配置するディレクトリの作成
!ls drive/colab/ImageClassifier/training_data

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option
kinoko_no_yama	takenoko_no_sato


# Turi Createでの画像分類

In [6]:
import turicreate as tc

# アノテーション
# data = tc.image_analysis.load_images('drive/colab/ImageClassifier/training_data', with_path=True)
# data['label'] = data['path'].apply(lambda path: 'kinoko_no_yama' if '/kinoko_no_yama' in path else 'takenoko_no_sato')
# data.save('training_data.sframe')
# data.explore()

# アノテーション済みのデータ読み込み
data =  tc.SFrame('drive/colab/ImageClassifier/training_data.sframe')
train_data, test_data = data.random_split(0.8)
model = tc.image_classifier.create(train_data, target='label')

# 評価
metrics = model.evaluate(test_data)
print(metrics['accuracy'])

# モデルの保存
# model.save('mymodel.model')

# mlmodelの書き出し
model.export_coreml('KinokonoyamaTakenokonosatoClassifier.mlmodel')




Download completed: /var/tmp/model_cache/resnet-50-symbol.json
Download completed: /var/tmp/model_cache/resnet-50-0000.params
Performing feature extraction on resized images...
Completed  64/157
Completed 128/157
Completed 157/157
PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Logistic regression:

--------------------------------------------------------

Number of examples          : 149

Number of classes           : 2

...

+-----------+----------+-----------+--------------+-------------------+---------------------+

Completed (Iteration limit reached).

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Performing feature extraction on resized images...
Completed 37/37
0.7297297297297297


/usr/local/lib/python3.6/dist-packages/coremltools/_deps/__init__.py:118: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  % (tensorflow.__version__, TF_MAX_VERSION))


# Web上の画像をフェッチしてテスト

In [27]:
!rm *.jpg
!wget https://images-na.ssl-images-amazon.com/images/I/9178rA%2BuFCL._SL1500_.jpg # きのこの山
# !wget https://images-na.ssl-images-amazon.com/images/I/91k0PeRaVmL._SL1500_.jpg # たけのこの里
image = tc.image_analysis.load_images('.', recursive=False)
model.predict(image)

--2018-06-21 08:50:53--  https://images-na.ssl-images-amazon.com/images/I/9178rA%2BuFCL._SL1500_.jpg
Resolving images-na.ssl-images-amazon.com (images-na.ssl-images-amazon.com)... 52.84.4.106
Connecting to images-na.ssl-images-amazon.com (images-na.ssl-images-amazon.com)|52.84.4.106|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 345471 (337K) [image/jpeg]
Saving to: '9178rA+uFCL._SL1500_.jpg'

9178rA+uFCL._SL1500 100%[===================>] 337.37K  --.-KB/s    in 0.04s   

2018-06-21 08:50:53 (8.97 MB/s) - '9178rA+uFCL._SL1500_.jpg' saved [345471/345471]



Unsupported image format. Supported formats are JPEG and PNG	 file: ./.rnd

Unsupported image format. Supported formats are JPEG and PNG	 file: ./9178rA+uFCL._SL1500_.jpg.1

Unsupported image format. Supported formats are JPEG and PNG	 file: ./9178rA+uFCL._SL1500_.jpg.2

Unsupported image format. Supported formats are JPEG and PNG	 file: ./KinokonoyamaTakenokonosatoClassifier.mlmodel

dtype: str
Rows: 1
['kinoko_no_yama']

# coremlモデルの書き出し

In [0]:
from google.colab import files
model.export_coreml('KinokonoyamaTakenokonosatoClassifier.mlmodel')
files.download('KinokonoyamaTakenokonosatoClassifier.mlmodel')